In [1]:
from pathlib import Path
from src.github_miner.linker.transformers import GitHubProjectTransformer
from src.common.models import GitProject, Issue
from src.inspector_git.linker.transformers import GitProjectTransformer
from src.inspector_git.reader.iglog.readers.ig_log_reader import IGLogReader
from src.run_blame import check_blame
import sys
import pickle
from src.jira_miner.reader_dto.loader import JiraJsonLoader
from src.jira_miner.linker.transformers import JiraProjectTransformer
from src.github_miner.reader_dto.loader import GithubJsonLoader
from src.github_miner.linker.transformers import GitHubProjectTransformer

# sys.setrecursionlimit(16336)


def build_and_check_project(iglog_path: str, repo_path: str, compute_annotated_lines :bool) -> GitProject:
    iglog_file = Path(iglog_path)

    with open(iglog_file, "r", encoding="utf-8") as f:
        git_log_dto = IGLogReader().read(f)

    transformer = GitProjectTransformer(
        git_log_dto,
        name=iglog_file.stem,
        compute_annotated_lines=compute_annotated_lines,
    )
    project = transformer.transform()

    if compute_annotated_lines:
        check_blame(project, repo_path)

    return project

git_project = build_and_check_project(
    "../../test-input/inspector-git/zeppelin.iglog",
    "/home/vortex/Work/BachelorThesis/voyager-target/test_inspector_git_repo/TestInspectorGitRepo",
    False
)

In [2]:
path_jira = "../../test-input/jira-miner/ZEPPELIN-detailed-issues.json"

jira_loader = JiraJsonLoader(path_jira)
jira_data = jira_loader.load()

jira_transformer = JiraProjectTransformer(jira_data, name="Jira Project")
jira_project = jira_transformer.transform()

In [3]:
path_github = "../../test-input/github-miner/githubProject.json"

github_loader = GithubJsonLoader(path_github)
github_data = github_loader.load()

github_transformer = GitHubProjectTransformer(github_data, name="GitHub Project")
github_project = github_transformer.transform()

In [6]:
from src.common.project_linkers import ProjectLinker

ProjectLinker.link_projects(github_project, jira_project, jira_data)
ProjectLinker.link_projects(jira_project, git_project)
ProjectLinker.link_projects(github_project, git_project)

[Linker] Direct PR–Commit links: 916
